implementation of chatbot using NLP

In [ ]:
pip install nltk scikit-learn streamlit

  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 1.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 794.0 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 249.6 kB/s eta 0:00:00a 0:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 690.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 482.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.1/32.1 MB 795.0 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 768.7 kB/s eta 0:00:0000:0100:01
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)


In [1]:
# importing libraries
import nltk
import random
import os
import ssl
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
ssl._create_default_https_context =ssl._create_unverified_context
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/aasthabaid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [28]:
intents = [
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "What's up", "Good day"],
        "responses": ["Hi there! Ready to talk cricket?", "Hello! Let's discuss cricket.", "Hey! What’s on your mind about cricket?"]
    },
    {
        "tag" : "cricket",
        "patterns" : ["what is cricket" , "how is cricket played"],
        "responses" :["Cricket is a bat-and-ball sport played between two teams of 11 players"," One side bats while the other fields, just as in baseball, and similarly one side attempts to score runs while the other side tries to stop them, and the side with the most runs wins."]
                      
    },
    {
        "tag": "rules",
        "patterns": ["What are the rules of cricket?", "Explain cricket rules", "How is cricket played?"],
        "responses": ["Cricket is played between two teams of 11 players. One team bats, trying to score runs, while the other bowls and fields to dismiss the batters. The team with the most runs wins."]
    },
    {
        "tag": "formats",
        "patterns": ["What are the formats of cricket?", "Types of cricket matches?", "What is Test, ODI, and T20?"],
        "responses": ["Cricket has three main formats: Test (5-day matches), One Day Internationals (50 overs per side), and T20 (20 overs per side)."]
    },
    {
        "tag": "latest_match",
        "patterns": ["What is the latest cricket match?", "Who won the last cricket game?", "Give me the latest cricket score"],
        "responses": ["I don't have live data, but you can check ESPN, Cricbuzz, or ICC’s official website for the latest scores."]
    },
    {
        "tag": "greatest_players",
        "patterns": ["Who is the greatest cricketer?", "Best cricketer of all time?", "GOAT of cricket?"],
        "responses": ["There are many legends in cricket, including Sachin Tendulkar, Don Bradman, Virat Kohli, MS Dhoni, and Muttiah Muralitharan. It depends on the format and era!"]
    },
    {
        "tag": "world_cup",
        "patterns": ["When is the next Cricket World Cup?", "Who won the last Cricket World Cup?", "Cricket World Cup details?"],
        "responses": ["The ICC Cricket World Cup happens every four years. You can check ICC’s official site for details on the next one."]
    },
    {
        "tag": "ipl",
        "patterns": ["What is IPL?", "Tell me about the Indian Premier League", "Who won the last IPL?"],
        "responses": ["IPL (Indian Premier League) is a T20 cricket league in India featuring top international and Indian players. The latest results are available on IPL’s official website."]
    },
    {
        "tag": "batting",
        "patterns": ["What is batting in cricket?", "Who are the best batsmen?", "How do you become a great batsman?"],
        "responses": ["Batting is when a player tries to score runs by hitting the ball with a bat. Some of the best batsmen include Virat Kohli, Joe Root, Steve Smith, and Babar Azam."]
    },
    {
        "tag": "bowling",
        "patterns": ["What is bowling in cricket?", "Who are the best bowlers?", "How do you become a great bowler?"],
        "responses": ["Bowling is when a player delivers the ball towards the batsman to get them out. Great bowlers include Muttiah Muralitharan, Dale Steyn, Glenn McGrath, and Jasprit Bumrah."]
    },
    {
        "tag": "fielding",
        "patterns": ["What is fielding in cricket?", "Who is the best fielder?", "How important is fielding in cricket?"],
        "responses": ["Fielding is stopping runs and catching batsmen out. Great fielders include Jonty Rhodes, Ravindra Jadeja, and AB de Villiers."]
    },
    {
        "tag": "umpire",
        "patterns": ["What does an umpire do?", "How does the umpire make decisions?", "What are the signals of an umpire?"],
        "responses": ["An umpire enforces the rules of cricket, makes decisions on dismissals, and signals boundaries, no-balls, and wides."]
    },
    {
        "tag": "drs",
        "patterns": ["What is DRS in cricket?", "How does the Decision Review System work?", "What are the rules of DRS?"],
        "responses": ["DRS (Decision Review System) allows teams to challenge an umpire’s decision using technology like ball-tracking and UltraEdge to check for errors."]
    },
    {
        "tag": "powerplay",
        "patterns": ["What is powerplay in cricket?", "How does powerplay work?", "Powerplay rules?"],
        "responses": ["In limited-overs cricket, the powerplay restricts fielding placements. In ODIs, the first 10 overs allow only two fielders outside the 30-yard circle."]
    },
    {
        "tag": "fastest_centuries",
        "patterns": ["Who scored the fastest century?", "Fastest 100 in cricket history?", "Who holds the fastest century record?"],
        "responses": ["AB de Villiers holds the record for the fastest ODI century (31 balls). The fastest Test century was by Brendon McCullum (54 balls)."]
    },
    {
        "tag": "icc_rankings",
        "patterns": ["What are the latest ICC rankings?", "Who is the top-ranked batsman?", "Which team is number one in ICC rankings?"],
        "responses": ["ICC updates rankings frequently. You can check ICC’s official website for the latest rankings in Tests, ODIs, and T20s."]
    },
    {
        "tag": "net_run_rate",
        "patterns": ["What is net run rate in cricket?", "How is NRR calculated?", "Explain net run rate"],
        "responses": ["Net Run Rate (NRR) is calculated as (total runs scored per over) - (total runs conceded per over). It is used in tournaments to break ties in points."]
    },
    {
        "tag": "underdog_teams",
        "patterns": ["Which are the best underdog teams in cricket?", "Has an underdog team won the World Cup?", "Surprise teams in cricket history?"],
        "responses": ["Some memorable underdog stories include Kenya’s 2003 World Cup semi-final run, Afghanistan’s rise in cricket, and Bangladesh defeating top teams."]
    },
    {
        "tag": "equipment",
        "patterns": ["What equipment is used in cricket?", "What is a cricket bat made of?", "How is a cricket ball made?"],
        "responses": ["Cricket requires a bat (made of willow wood), a ball (made of leather), pads, gloves, helmets, and wickets (stumps and bails)."]
    },
    {
        "tag": "unexpected",
        "patterns": ["Tell me a fun fact about cricket", "Weird cricket rules?", "Most bizarre cricket moment?"],
        "responses": ["A fun fact: Cricket matches can last up to 5 days! Also, a match once lasted 12 days but ended as a draw."]
    },
    {
        "tag": "farewell",
        "patterns": ["Bye", "See you later", "Goodbye"],
        "responses": ["Goodbye! Come back anytime to talk cricket!", "See you later, cricket fan!", "Take care, and keep enjoying cricket!"]
    }
]


In [29]:
vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', ngram_range=(1, 2))
clf = LogisticRegression(random_state=0, max_iter=10000)

tags = []
patterns = []

for intent in intents:
    for pattern in intent['patterns']:
        tags.append(intent['tag'])  # Corrected the dictionary access method
        patterns.append(pattern)

In [30]:
x = vectorizer.fit_transform(patterns)
y = tags
clf.fit(x,y)

LogisticRegression(max_iter=10000, random_state=0)

In [31]:
def chatbot(input_text):
    input_text = vectorizer.transform([input_text])  # Convert input text into vector form
    tag = clf.predict(input_text)[0]  # Predict the tag


    for intent in intents:
        if intent['tag'] == tag:
            return random.choice(intent['responses'])  # Return a random response
    
    return "I'm sorry, I don't understand." 

In [32]:
user_input = "Hello"
response = chatbot(user_input)
print(response)

Hello! Let's discuss cricket.


In [33]:
user_input = "whats cricket"
response = chatbot(user_input)
print(response)

Cricket is a bat-and-ball sport played between two teams of 11 players


In [34]:
user_input = "i need help"
response = chatbot(user_input)
print(response)

Hey! What’s on your mind about cricket?


In [35]:
user_input = "how many innings are there in a match"
response = chatbot(user_input)
print(response)

Hey! What’s on your mind about cricket?
